<a href="https://colab.research.google.com/github/fabriciocgf/fcc_machine_learning/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-01-28 13:34:31--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  39.2MB/s    in 0.6s    

2022-01-28 13:34:32 (39.2 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# filter data
df_books_cnt = pd.DataFrame(df_ratings.groupby('isbn').size(), columns=['count'])
popular_books = list(set(df_books_cnt.query('count >= 100').index))  # noqa
books_filter = df_ratings.isbn.isin(popular_books).values

df_users_cnt = pd.DataFrame(df_ratings.groupby('user').size(), columns=['count'])
active_users = list(set(df_users_cnt.query('count >= 200').index))  # noqa
users_filter = df_ratings.user.isin(active_users).values
#df_ratings_filtered = df_ratings[users_filter]
df_ratings_filtered = df_ratings[books_filter & users_filter]
df_ratings_filtered.shape

(49781, 3)

In [ ]:
df_book_features = df_ratings_filtered.pivot(
    index='isbn',
    columns='user',
    values='rating'
).fillna(0)
df_book_features.shape

(731, 888)

In [ ]:
from scipy.sparse import csr_matrix
# convert dataframe of movie features to scipy sparse matrix
mat_book_features = csr_matrix(df_book_features.values)

In [ ]:
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)

In [ ]:
hashmap = df_books.set_index('isbn').filter(items = list(df_book_features.index), axis=0).to_dict()['title']
hashmap1 = dict()
for key, value in hashmap.items():
  hashmap1.setdefault(value, list()).append(key)
print(hashmap1)

{'Politically Correct Bedtime Stories: Modern Tales for Our Life and Times': ['002542730X'], 'Angels': ['0060008032'], 'The Boy Next Door': ['0060096195'], 'Men Are from Mars, Women Are from Venus: A Practical Guide for Improving Communication and Getting What You Want in Your Relationships': ['006016848X'], 'Divine Secrets of the Ya-Ya Sisterhood : A Novel': ['0060173289'], 'The Poisonwood Bible': ['0060175400'], "Daughter of Fortune : A Novel (Oprah's Book Club (Hardcover))": ['006019491X'], 'Prodigal Summer': ['0060199652'], "I Know This Much Is True (Oprah's Book Club)": ['0060391626'], 'Stupid White Men ...and Other Sorry Excuses for the State of the Nation!': ['0060392452'], 'The Divine Secrets of the Ya-Ya Sisterhood: A Novel': ['0060502258'], 'The Bean Trees': ['0060915544', '0061097314'], 'Their Eyes Were Watching God: A Novel': ['0060916508'], 'Animal Dreams': ['0060921145'], 'Pigs in Heaven': ['0060922532'], 'Divine Secrets of the Ya-Ya Sisterhood: A Novel': ['0060928336'], 

In [ ]:
hashmap = df_books.set_index('isbn').filter(items = list(df_book_features.index), axis=0).to_dict()['title']
hashmap1 = dict(map(reversed, hashmap.items()))
hashmap2 = list(df_book_features.index)
print(hashmap2.index(hashmap1["Where the Heart Is (Oprah's Book Club (Paperback))"]))
print(hashmap[hashmap2[435]])

435
Where the Heart Is (Oprah's Book Club (Paperback))


In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  model.fit(mat_book_features)
  book_code = hashmap2.index(hashmap1[book])
  recommendation = model.kneighbors(mat_book_features[book_code],n_neighbors=6)
  list_books = []
  for idx, book_index in enumerate(recommendation[1][0]):
    test = [hashmap[hashmap2[book_index]], recommendation[0][0][idx]]
    list_books.append(test)
  recommended_books = book, list_books[2:]
  return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

("Where the Heart Is (Oprah's Book Club (Paperback))", [['I Know This Much Is True', 0.7677075], ['The Surgeon', 0.7699411], ['The Weight of Water', 0.77085835], ["I'll Be Seeing You", 0.8016211]])
You passed the challenge! 🎉🎉🎉🎉🎉


## Implementing a fuzzy finder in our database of books

In [ ]:
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
def match(book):
  match_tuple = []
  # get match
  for title, isbn in hashmap1.items():
    ratio = fuzz.ratio(title.lower(), book.lower())
    if ratio >= 50:
      match_tuple.append((title, ratio))
  # sort
  match_tuple = sorted(match_tuple, key=lambda x: x[1])[::-1]
  if not match_tuple:
    print('Oops! No match is found')
  else:
    print('Found possible matches in our database: ' '{0}\n'.format([x[0] for x in match_tuple]))
  return match_tuple[0][0]

In [ ]:
match("the Life of pi")

Found possible matches in our database: ['Life of Pi', 'The Face of Deception', 'The Prince of Tides', 'The Secret Life of Bees', 'The Loop', 'The Alienist', 'The Tao of Pooh', 'The Color of Magic', 'The Burden of Proof', 'The Valley of Horses', 'The House of the Spirits', 'The King of Torts', 'The Client', 'The Temple of My Familiar', 'The English Patient', 'The Reef', 'The Firm', 'The Key to Midnight', 'The Mists of Avalon', 'The Weight of Water', 'The Bonfire of the Vanities', 'The Eyes of Darkness', 'The House of Thunder', "The Pilot's Wife", 'The Pelican Brief', 'The Last Precinct', 'The Bridges of Madison County', 'The Rapture of Canaan', 'The Angel of Darkness', 'Peace Like a River', 'The Green Mile', 'From the Corner of His Eye', 'The Bluest Eye', 'The Third Twin', 'P Is for Peril', 'The Winner', 'The Accidental Tourist', 'The Last Juror', 'The Blind Assassin', 'The Lost World', "The Pilot's Wife : A Novel"]



'Life of Pi'

In [ ]:
books = get_recommends(match("the Life of pi"))
print(books)

Found possible matches in our database: ['Life of Pi', 'The Face of Deception', 'The Prince of Tides', 'The Secret Life of Bees', 'The Loop', 'The Alienist', 'The Tao of Pooh', 'The Color of Magic', 'The Burden of Proof', 'The Valley of Horses', 'The House of the Spirits', 'The King of Torts', 'The Client', 'The Temple of My Familiar', 'The English Patient', 'The Reef', 'The Firm', 'The Key to Midnight', 'The Mists of Avalon', 'The Weight of Water', 'The Bonfire of the Vanities', 'The Eyes of Darkness', 'The House of Thunder', "The Pilot's Wife", 'The Pelican Brief', 'The Last Precinct', 'The Bridges of Madison County', 'The Rapture of Canaan', 'The Angel of Darkness', 'Peace Like a River', 'The Green Mile', 'From the Corner of His Eye', 'The Bluest Eye', 'The Third Twin', 'P Is for Peril', 'The Winner', 'The Accidental Tourist', 'The Last Juror', 'The Blind Assassin', 'The Lost World', "The Pilot's Wife : A Novel"]

('Life of Pi', [['The Four Agreements: A Practical Guide to Personal 